In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights.
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        return ones_p / p
    
    w_star = numerator / denominator
    return w_star

def mv_weights(Theta_hat, mu, target_return=0.01):
    """
    Compute Mean-Variance portfolio weights with target return.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected returns
    target_return : float
        Target portfolio return (default: 0.01 = 1% monthly)
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute key quantities
    A = ones_p @ Theta_hat @ ones_p
    B = ones_p @ Theta_hat @ mu
    C = mu @ Theta_hat @ mu
    D = A * C - B * B
    
    # Check for singularity
    if np.abs(D) < 1e-10:
        if np.abs(A) > 1e-10:
            w_star = (Theta_hat @ ones_p) / A
            return w_star
        else:
            return ones_p / p
    
    # Compute Lagrange multipliers
    lambda1 = (C - B * target_return) / D
    lambda2 = (A * target_return - B) / D
    
    # Compute weights
    w_star = lambda1 * (Theta_hat @ ones_p) + lambda2 * (Theta_hat @ mu)
    
    return w_star


def msr_weights(Theta_hat, mu):
    """
    Compute Maximum Sharpe Ratio portfolio weights.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected excess returns
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights (sum to 1)
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute unnormalized weights
    w_unnorm = Theta_hat @ mu
    
    # Normalize to sum to 1
    weight_sum = np.sum(w_unnorm)
    
    if np.abs(weight_sum) < 1e-10:
        return ones_p / p
    
    w_star = w_unnorm / weight_sum
    
    return w_star


import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import time
from decimal import Decimal, ROUND_HALF_UP

import scipy.linalg as sl
def afm_est(Y, NF):

    n = Y.shape[0]
    p = Y.shape[1]
    
    # L'L normalization
    ev_ = sl.eigh(np.cov(Y))

    # Sort eigenvalues in descending order
    indx_ev = ev_[0].argsort()[::-1]
    # Get eigenvectors
    evec = ev_[1][:, indx_ev]

    # Determining Factors
    F = np.sqrt(n) * evec[:, 0:NF]

    # Factorloadings
    L = Y.T @ F/n

    """
    # F'F normalization
    ev_ = sl.eigh(np.cov(Y.T))

    # Sort eigenvalues in descending order
    indx_ev = ev_[0].argsort()[::-1]
    # Get eigenvectors
    evec = ev_[1][:, indx_ev]

    # Determining Factors
    L = np.sqrt(p) * evec[:, 0:NF]
    # Factorloadings
    F = Y @ L/p
    """
    resd = Y - F @ L.T

    ret_ = {'F': F, 'L': L, 'resd': resd}

    return ret_

def nf_bn(Y, nf_max):

    n = Y.shape[0]
    p = Y.shape[1]

    IC1 = np.empty((nf_max, 1))
    IC1[:] = np.nan
    IC2 = np.empty((nf_max, 1))
    IC2[:] = np.nan
    IC3 = np.empty((nf_max, 1))
    IC3[:] = np.nan

    for ii in range(1, nf_max+1):
        ret_afm = afm_est(Y, ii)

        V = np.mean(ret_afm['resd']**2)
        
        #Information criteria
        #IC1
        IC1[ii-1, 0] = np.log(V) + ii * ((p+n)/(p*n) * np.log((p*n)/(p+n)))
        #IC2
        IC2[ii-1, 0] = np.log(V) + ii * ((p+n)/(p*n) * np.log(min(n,p)))
        #IC3
        IC3[ii-1, 0] = np.log(V) + ii * (np.log(min(n,p)) / (min(n,p)))
    
    ICs = np.empty((3, 2))
    ICs[:] = np.nan

    ICs[0, 0] = IC1.argmin()
    ICs[1, 0] = IC2.argmin()
    ICs[2, 0] = IC3.argmin()

    ICs[0, 1] = IC1[int(ICs[0, 0])]
    ICs[1, 1] = IC2[int(ICs[1, 0])]
    ICs[2, 1] = IC3[int(ICs[2, 0])]

    ret_ = {'num_f': ICs[:,0], 'ICs': ICs[:,1]}

    return ret_

def soft_t(z, a):
  t1 = np.sign(z)
  b = np.abs(z) - a
  t2 = b * (b >= 0)
  z_t = t1 * t2
  return z_t

def cov_e_poet(resd, C, N, T):
    rate_thres = 1/np.sqrt(N) + np.sqrt((np.log(N))/T)
    # lam = rate_thres * C * np.ones(shape=(N,N))
    
    sig_e_samp = np.cov(resd.T)
    
    
    thet_par = np.empty((N, N))
    thet_par[:] = np.nan
    
    for ii in range(0, N):
        for jj in range(0, N):
            thet_par[ii, jj] = np.mean((resd[:, ii] * resd[:, jj] - sig_e_samp[ii, jj])**2)
    
    lam = rate_thres * C * np.sqrt(thet_par)
    
    """
    sig_e_diag=np.diag(np.sqrt(np.diag(sig_e_samp)))
    R = np.linalg.inv(sig_e_diag) @ sig_e_samp @ np.linalg.inv(sig_e_diag); 
    M = soft_t(R, lam)
    np.fill_diagonal(M, 1)
    sig_e_hat = sig_e_diag @ M @ sig_e_diag
    """

    sig_e_diag = np.diag(sig_e_samp)
    sig_e_hat = soft_t(sig_e_samp, lam)
    np.fill_diagonal(sig_e_hat, sig_e_diag)

    return sig_e_hat

def poet(Y_star,NF_max):
    n = Y_star.shape[0]
    p = Y_star.shape[1]
    
    num_f = nf_bn(Y_star, NF_max)['num_f']
    num_f = int(num_f[0])
    est_afm = afm_est(Y_star, num_f)

    F = est_afm['F']
    L = est_afm['L']
    resd = est_afm['resd']
    sigma_u_hat = cov_e_poet(resd, 2, p, n)
    sigma_u_hat_inv = np.linalg.inv(sigma_u_hat)
    
    A=L.T @ sigma_u_hat_inv @ L
    I=np.eye(A.shape[0])
    
    Theta_hat = sigma_u_hat_inv - sigma_u_hat_inv @ L @ np.linalg.inv(I+A) @ L.T @ sigma_u_hat_inv
    return Theta_hat


def load_finbert_signals(signals_path):
    """
    Load FinBERT monthly signals from CSV file.
    
    Parameters:
    -----------
    signals_path : str
        Path to monthly_signals.csv file
    
    Returns:
    --------
    signals_df : pd.DataFrame
        DataFrame with columns: symbol, company, year_month, signal, avg_sentiment_score
    """
    try:
        signals_df = pd.read_csv(signals_path)
        # Convert year_month to datetime (end of month)
        signals_df['date'] = pd.to_datetime(signals_df['year_month']) + pd.offsets.MonthEnd(0)
        return signals_df
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load FinBERT signals: {e}")
        return pd.DataFrame(columns=['symbol', 'company', 'year_month', 'signal', 'date'])


def get_buy_signal_permnos_for_date(signals_df, ticker_to_permno, date):
    """
    Get set of permnos with 'buy' or 'sell' signals for a specific date.
    
    Parameters:
    -----------
    signals_df : pd.DataFrame
        FinBERT signals dataframe
    ticker_to_permno : dict
        Mapping from ticker symbol to permno
    date : pd.Timestamp
        Date to get signals for
    
    Returns:
    --------
    permno_set : set
        Set of permnos with buy or sell signals on this date
    """
    # Get signals for this date
    date_signals = signals_df[signals_df['date'] == date]
    
    # Filter for buy and sell signals only (exclude hold)
    buy_signals = date_signals[date_signals['signal'] == 'buy']
    sell_signals = date_signals[date_signals['signal'] == 'sell']
    
    # Convert tickers to permnos
    permnos = set()
    for ticker in buy_signals['symbol'].values:
        if ticker in ticker_to_permno:
            permnos.add(ticker_to_permno[ticker])
    for ticker in sell_signals['symbol'].values:
        if ticker in ticker_to_permno:
            permnos.add(ticker_to_permno[ticker])
    
    return permnos


def create_ticker_to_permno_mapping(df):
    """
    Create a mapping from ticker to permno from the returns dataframe.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Returns dataframe with 'ticker' and 'permno' columns
    
    Returns:
    --------
    ticker_to_permno : dict
        Mapping from ticker to permno (uses most recent permno for each ticker)
    """
    if 'ticker' not in df.columns:
        raise ValueError("DataFrame must have 'ticker' column for mapping")
    
    # Drop NaN tickers
    valid_df = df[df['ticker'].notna()].copy()
    
    # Get the most recent permno for each ticker
    ticker_to_permno = valid_df.groupby('ticker')['permno'].last().to_dict()
    
    return ticker_to_permno


def calculate_exit_transaction_cost(prev_weights_dict, prev_oos_returns_dict, 
                                    prev_gross_return, transaction_cost, verbose=False):
    """
    Calculate transaction cost when exiting the market (liquidating all positions).
    Enforces Immediate Liquidation logic:
    - Next period return is 0.0 (Cash)
    - Cost is paid on current portfolio value
    """
    if len(prev_weights_dict) == 0:
        return 0.0, 0.0, 0.0
    
    # Step 1: Adjust previous weights to current period's BEGINNING (drift from t-1 to t)
    adjusted_prev = {}
    for asset, prev_w in prev_weights_dict.items():
        if asset in prev_oos_returns_dict:
            prev_r = prev_oos_returns_dict[asset]
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
        else:
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
    
    # Step 2: Turnover (Selling everything to Cash)
    turnover = sum(abs(w) for w in adjusted_prev.values())
    
    # Step 3: Cost 
    # Paper Formula: c * (1 + R_next) * Turnover. 
    # Since R_next (Cash) is 0.0, this simplifies to c * 1.0 * Turnover.
    tc = transaction_cost * 1.0 * turnover
    
    # Step 4: Net Return is 0.0 (Cash return) - Cost
    net_return = -tc
    
    if verbose:
        print(f"  Liquidating positions | Turnover: {turnover:>6.4f} | TC: {tc:>8.6f}")
    
    return turnover, tc, net_return


def backtest_dnn_finbert(df, 
                         data_factor,
                         signals_path='monthly_signals.csv',
                         test_start_date='2020-01-31', 
                         test_end_date='2024-11-30',
                         lookback_window=180,
                         transaction_cost=0.001,
                         architecture=5,
                         const_err_cov=2.5,
                         use_CV_err=False,
                         target_return=0.01,
                         verbose=True):
    """
    Backtest DNN-FM + FinBERT signals using GMV, MV, and MSR strategies.
    Records zero returns and empty weights when skipping periods.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ticker, ret_fwd_1
    data_factor : pd.DataFrame
        Factor data for DNN-FM model (indexed by date)
    signals_path : str
        Path to monthly_signals.csv file
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.001 = 10 bps)
    architecture : int
        DNN architecture number (default: 5)
    const_err_cov : float
        Constant for error covariance thresholding (default: 2.5)
    use_CV_err : bool
        Whether to use cross-validation for hyperparameters
    target_return : float
        Target return for MV portfolio (default: 0.01 = 1% monthly)
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results_df : pd.DataFrame (GMV)
    metrics : dict (GMV)
    results_df_2 : pd.DataFrame (MV)
    metrics_2 : dict (MV)
    results_df_3 : pd.DataFrame (MSR)
    metrics_3 : dict (MSR)
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Create ticker to permno mapping
    if verbose:
        print("Creating ticker to permno mapping...")
    ticker_to_permno = create_ticker_to_permno_mapping(df)
    if verbose:
        print(f"Mapped {len(ticker_to_permno)} unique tickers to permnos")
    
    # Load FinBERT signals
    if verbose:
        print(f"Loading FinBERT signals from {signals_path}...")
    signals_df = load_finbert_signals(signals_path)
    if len(signals_df) == 0:
        raise ValueError("No FinBERT signals loaded")
    
    if verbose:
        print(f"Loaded {len(signals_df)} monthly signals")
        print(f"Signal distribution:")
        print(signals_df['signal'].value_counts())
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results - GMV
    portfolio_returns = []
    portfolio_dates = []
    portfolio_weights_list = []
    portfolio_turnover_list = []
    portfolio_gross_returns = []
    
    # Storage for results - MV
    portfolio_returns_2 = []
    portfolio_dates_2 = []
    portfolio_weights_list_2 = []
    portfolio_turnover_list_2 = []
    portfolio_gross_returns_2 = []
    
    # Storage for results - MSR
    portfolio_returns_3 = []
    portfolio_dates_3 = []
    portfolio_weights_list_3 = []
    portfolio_turnover_list_3 = []
    portfolio_gross_returns_3 = []
    
    # Track weights by permno - GMV
    prev_weights_dict = {}
    prev_oos_returns_dict = {}
    prev_gross_return = 0.0
    
    # Track weights by permno - MV
    prev_weights_dict_2 = {}
    prev_oos_returns_dict_2 = {}
    prev_gross_return_2 = 0.0
    
    # Track weights by permno - MSR
    prev_weights_dict_3 = {}
    prev_oos_returns_dict_3 = {}
    prev_gross_return_3 = 0.0
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print("STARTING BACKTEST WITH DNN-FM + FINBERT (GMV/MV/MSR)")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        
        # Get buy/sell signal permnos for current date
        allowed_permnos = get_buy_signal_permnos_for_date(
            signals_df, ticker_to_permno, current_date
        )
        
        # ========================================
        # CRITICAL: Get OOS returns FIRST before any early exits
        # ========================================
        oos_data = df[(df['datadate'] == current_date) & (df['permno'].isin(allowed_permnos))]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Now handle early exit cases
        if len(allowed_permnos) == 0:
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ No signals for {current_date.strftime('%Y-%m-%d')}, recording zero return")
            
            # GMV
            turnover, tc, net_return = calculate_exit_transaction_cost(
                prev_weights_dict, prev_oos_returns_dict, prev_gross_return, 
                transaction_cost, verbose=verbose
            )
            portfolio_returns.append(net_return)
            portfolio_dates.append(current_date)
            portfolio_weights_list.append({})
            portfolio_turnover_list.append(turnover)
            portfolio_gross_returns.append(0.0)
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            
            # MV
            turnover_2, tc_2, net_return_2 = calculate_exit_transaction_cost(
                prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2,
                transaction_cost, verbose=False
            )
            portfolio_returns_2.append(net_return_2)
            portfolio_dates_2.append(current_date)
            portfolio_weights_list_2.append({})
            portfolio_turnover_list_2.append(turnover_2)
            portfolio_gross_returns_2.append(0.0)
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            
            # MSR
            turnover_3, tc_3, net_return_3 = calculate_exit_transaction_cost(
                prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3,
                transaction_cost, verbose=False
            )
            portfolio_returns_3.append(net_return_3)
            portfolio_dates_3.append(current_date)
            portfolio_weights_list_3.append({})
            portfolio_turnover_list_3.append(turnover_3)
            portfolio_gross_returns_3.append(0.0)
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            
            continue
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window, filtered by allowed permnos
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date) &
                        (df['permno'].isin(allowed_permnos))]
        
        # Pivot to get returns matrix (time x assets)
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        # Filter assets with any NaNs in this window
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f"  FinBERT signals: {len(allowed_permnos)} | Assets w/ data: {p_current}")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), recording zero return")
            
            # GMV
            turnover, tc, net_return = calculate_exit_transaction_cost(
                prev_weights_dict, prev_oos_returns_dict, prev_gross_return,
                transaction_cost, verbose=verbose
            )
            portfolio_returns.append(net_return)
            portfolio_dates.append(current_date)
            portfolio_weights_list.append({})
            portfolio_turnover_list.append(turnover)
            portfolio_gross_returns.append(0.0)
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            
            # MV
            turnover_2, tc_2, net_return_2 = calculate_exit_transaction_cost(
                prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2,
                transaction_cost, verbose=False
            )
            portfolio_returns_2.append(net_return_2)
            portfolio_dates_2.append(current_date)
            portfolio_weights_list_2.append({})
            portfolio_turnover_list_2.append(turnover_2)
            portfolio_gross_returns_2.append(0.0)
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            
            # MSR
            turnover_3, tc_3, net_return_3 = calculate_exit_transaction_cost(
                prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3,
                transaction_cost, verbose=False
            )
            portfolio_returns_3.append(net_return_3)
            portfolio_dates_3.append(current_date)
            portfolio_weights_list_3.append({})
            portfolio_turnover_list_3.append(turnover_3)
            portfolio_gross_returns_3.append(0.0)
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            
            continue
        
        try:
            # Get factor data for the same window
            train_factor = data_factor.loc[window_start_date:window_end_date]
            
            # Ensure factor data aligns with returns data
            train_factor = train_factor.reindex(index=window_dates)
            
            if train_factor.isna().any().any():
                raise ValueError("Factor data contains NaN values in training window")
            
            F = train_factor.values.astype(float)
            
            # Demean the returns
            Y_bar = Y.mean(axis=0)
            Y_star = Y - Y_bar
            
            if verbose:
                print(f"  Running POET (arch={architecture})...")
            
            # Run poet
            Theta_hat = poet(Y_star, 8)
            
            if verbose:
                print(f"  Computing GMV weights...")
            w_star = gmv_weights(Theta_hat)
            
            if verbose:
                print(f"  Computing MV weights...")
            w_star_2 = mv_weights(Theta_hat, Y_bar, target_return=target_return)
            
            if verbose:
                print(f"  Computing MSR weights...")
            w_star_3 = msr_weights(Theta_hat, Y_bar)
            
            # Create weights dictionaries
            new_weights_dict = {asset: w_star[i] for i, asset in enumerate(current_assets)}
            new_weights_dict_2 = {asset: w_star_2[i] for i, asset in enumerate(current_assets)}
            new_weights_dict_3 = {asset: w_star_3[i] for i, asset in enumerate(current_assets)}
            
        except Exception as e:
            if verbose:
                print(f"  ✗ Error: {e}")
                print(f"  Recording zero return")
            
            # GMV
            turnover, tc, net_return = calculate_exit_transaction_cost(
                prev_weights_dict, prev_oos_returns_dict, prev_gross_return,
                transaction_cost, verbose=verbose
            )
            portfolio_returns.append(net_return)
            portfolio_dates.append(current_date)
            portfolio_weights_list.append({})
            portfolio_turnover_list.append(turnover)
            portfolio_gross_returns.append(0.0)
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            
            # MV
            turnover_2, tc_2, net_return_2 = calculate_exit_transaction_cost(
                prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2,
                transaction_cost, verbose=False
            )
            portfolio_returns_2.append(net_return_2)
            portfolio_dates_2.append(current_date)
            portfolio_weights_list_2.append({})
            portfolio_turnover_list_2.append(turnover_2)
            portfolio_gross_returns_2.append(0.0)
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            
            # MSR
            turnover_3, tc_3, net_return_3 = calculate_exit_transaction_cost(
                prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3,
                transaction_cost, verbose=False
            )
            portfolio_returns_3.append(net_return_3)
            portfolio_dates_3.append(current_date)
            portfolio_weights_list_3.append({})
            portfolio_turnover_list_3.append(turnover_3)
            portfolio_gross_returns_3.append(0.0)
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            
            continue

        # Normalize weights to sum to 1 - GMV
        weight_sum = sum(new_weights_dict.values())
        if weight_sum > 1e-10:
            new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
        else:
            if verbose:
                print("  ⚠ GMV: Zero weight sum, recording zero return")
            
            # GMV
            turnover, tc, net_return = calculate_exit_transaction_cost(
                prev_weights_dict, prev_oos_returns_dict, prev_gross_return,
                transaction_cost, verbose=verbose
            )
            portfolio_returns.append(net_return)
            portfolio_dates.append(current_date)
            portfolio_weights_list.append({})
            portfolio_turnover_list.append(turnover)
            portfolio_gross_returns.append(0.0)
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            
            # MV
            turnover_2, tc_2, net_return_2 = calculate_exit_transaction_cost(
                prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2,
                transaction_cost, verbose=False
            )
            portfolio_returns_2.append(net_return_2)
            portfolio_dates_2.append(current_date)
            portfolio_weights_list_2.append({})
            portfolio_turnover_list_2.append(turnover_2)
            portfolio_gross_returns_2.append(0.0)
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            
            # MSR
            turnover_3, tc_3, net_return_3 = calculate_exit_transaction_cost(
                prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3,
                transaction_cost, verbose=False
            )
            portfolio_returns_3.append(net_return_3)
            portfolio_dates_3.append(current_date)
            portfolio_weights_list_3.append({})
            portfolio_turnover_list_3.append(turnover_3)
            portfolio_gross_returns_3.append(0.0)
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            
            continue
        
        # Normalize weights to sum to 1 - MV
        weight_sum_2 = sum(new_weights_dict_2.values())
        if weight_sum_2 > 1e-10:
            new_weights_dict_2 = {k: v/weight_sum_2 for k, v in new_weights_dict_2.items()}
        else:
            if verbose:
                print("  ⚠ MV: Zero weight sum, using GMV weights")
            new_weights_dict_2 = new_weights_dict.copy()
        
        # Normalize weights to sum to 1 - MSR
        weight_sum_3 = sum(new_weights_dict_3.values())
        if weight_sum_3 > 1e-10:
            new_weights_dict_3 = {k: v/weight_sum_3 for k, v in new_weights_dict_3.items()}
        else:
            if verbose:
                print("  ⚠ MSR: Zero weight sum, using GMV weights")
            new_weights_dict_3 = new_weights_dict.copy()
        
        # --- 3. OOS Returns & Transaction Costs ---
        
        # Find common assets between weights and returns
        common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
        common_assets_2 = set(new_weights_dict_2.keys()) & set(oos_returns_dict.keys())
        common_assets_3 = set(new_weights_dict_3.keys()) & set(oos_returns_dict.keys())
        
        if len(common_assets) == 0:
            if verbose:
                print("  ⚠ No common assets with valid returns, recording zero return")
            
            # GMV
            turnover, tc, net_return = calculate_exit_transaction_cost(
                prev_weights_dict, prev_oos_returns_dict, prev_gross_return,
                transaction_cost, verbose=verbose
            )
            portfolio_returns.append(net_return)
            portfolio_dates.append(current_date)
            portfolio_weights_list.append({})
            portfolio_turnover_list.append(turnover)
            portfolio_gross_returns.append(0.0)
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            
            # MV
            turnover_2, tc_2, net_return_2 = calculate_exit_transaction_cost(
                prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2,
                transaction_cost, verbose=False
            )
            portfolio_returns_2.append(net_return_2)
            portfolio_dates_2.append(current_date)
            portfolio_weights_list_2.append({})
            portfolio_turnover_list_2.append(turnover_2)
            portfolio_gross_returns_2.append(0.0)
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            
            # MSR
            turnover_3, tc_3, net_return_3 = calculate_exit_transaction_cost(
                prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3,
                transaction_cost, verbose=False
            )
            portfolio_returns_3.append(net_return_3)
            portfolio_dates_3.append(current_date)
            portfolio_weights_list_3.append({})
            portfolio_turnover_list_3.append(turnover_3)
            portfolio_gross_returns_3.append(0.0)
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            
            continue
        
        # Filter to common assets and renormalize - GMV
        common_weights = {a: new_weights_dict[a] for a in common_assets}
        common_weight_sum = sum(common_weights.values())
        if common_weight_sum > 1e-10:
            common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
        else:
            if verbose:
                print("  ⚠ GMV: Zero weight sum after filtering, recording zero return")
            
            # GMV
            turnover, tc, net_return = calculate_exit_transaction_cost(
                prev_weights_dict, prev_oos_returns_dict, prev_gross_return,
                transaction_cost, verbose=verbose
            )
            portfolio_returns.append(net_return)
            portfolio_dates.append(current_date)
            portfolio_weights_list.append({})
            portfolio_turnover_list.append(turnover)
            portfolio_gross_returns.append(0.0)
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            
            # MV
            turnover_2, tc_2, net_return_2 = calculate_exit_transaction_cost(
                prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2,
                transaction_cost, verbose=False
            )
            portfolio_returns_2.append(net_return_2)
            portfolio_dates_2.append(current_date)
            portfolio_weights_list_2.append({})
            portfolio_turnover_list_2.append(turnover_2)
            portfolio_gross_returns_2.append(0.0)
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            
            # MSR
            turnover_3, tc_3, net_return_3 = calculate_exit_transaction_cost(
                prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3,
                transaction_cost, verbose=False
            )
            portfolio_returns_3.append(net_return_3)
            portfolio_dates_3.append(current_date)
            portfolio_weights_list_3.append({})
            portfolio_turnover_list_3.append(turnover_3)
            portfolio_gross_returns_3.append(0.0)
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            
            continue
        
        # Filter to common assets and renormalize - MV
        common_weights_2 = {a: new_weights_dict_2[a] for a in common_assets_2}
        common_weight_sum_2 = sum(common_weights_2.values())
        if common_weight_sum_2 > 1e-10:
            common_weights_2 = {k: v/common_weight_sum_2 for k, v in common_weights_2.items()}
        else:
            if verbose:
                print("  ⚠ MV: Zero weight sum after filtering, using GMV")
            common_weights_2 = common_weights.copy()
        
        # Filter to common assets and renormalize - MSR
        common_weights_3 = {a: new_weights_dict_3[a] for a in common_assets_3}
        common_weight_sum_3 = sum(common_weights_3.values())
        if common_weight_sum_3 > 1e-10:
            common_weights_3 = {k: v/common_weight_sum_3 for k, v in common_weights_3.items()}
        else:
            if verbose:
                print("  ⚠ MSR: Zero weight sum after filtering, using GMV")
            common_weights_3 = common_weights.copy()
        
        # Compute gross portfolio returns
        gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
        gross_return_2 = sum(common_weights_2[a] * oos_returns_dict[a] for a in common_assets_2)
        gross_return_3 = sum(common_weights_3[a] * oos_returns_dict[a] for a in common_assets_3)
        
        # Sanity checks
        if np.isnan(gross_return) or np.isinf(gross_return):
            if verbose:
                print(f"  ⚠ GMV: Invalid gross return: {gross_return}, recording zero return")
            
            # GMV
            turnover, tc, net_return = calculate_exit_transaction_cost(
                prev_weights_dict, prev_oos_returns_dict, prev_gross_return,
                transaction_cost, verbose=verbose
            )
            portfolio_returns.append(net_return)
            portfolio_dates.append(current_date)
            portfolio_weights_list.append({})
            portfolio_turnover_list.append(turnover)
            portfolio_gross_returns.append(0.0)
            prev_weights_dict = {}
            prev_oos_returns_dict = {}
            prev_gross_return = 0.0
            
            # MV
            turnover_2, tc_2, net_return_2 = calculate_exit_transaction_cost(
                prev_weights_dict_2, prev_oos_returns_dict_2, prev_gross_return_2,
                transaction_cost, verbose=False
            )
            portfolio_returns_2.append(net_return_2)
            portfolio_dates_2.append(current_date)
            portfolio_weights_list_2.append({})
            portfolio_turnover_list_2.append(turnover_2)
            portfolio_gross_returns_2.append(0.0)
            prev_weights_dict_2 = {}
            prev_oos_returns_dict_2 = {}
            prev_gross_return_2 = 0.0
            
            # MSR
            turnover_3, tc_3, net_return_3 = calculate_exit_transaction_cost(
                prev_weights_dict_3, prev_oos_returns_dict_3, prev_gross_return_3,
                transaction_cost, verbose=False
            )
            portfolio_returns_3.append(net_return_3)
            portfolio_dates_3.append(current_date)
            portfolio_weights_list_3.append({})
            portfolio_turnover_list_3.append(turnover_3)
            portfolio_gross_returns_3.append(0.0)
            prev_weights_dict_3 = {}
            prev_oos_returns_dict_3 = {}
            prev_gross_return_3 = 0.0
            
            continue
        
        # Calculate transaction costs - GMV
        if len(prev_weights_dict) > 0:
            # Step 1: Adjust ALL previous weights for returns
            adjusted_prev = {}
            
            for asset, prev_w in prev_weights_dict.items():
                if asset in prev_oos_returns_dict:
                    prev_r = prev_oos_returns_dict[asset]
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
                else:
                    # Asset had weight but no return data
                    if abs(1 + prev_gross_return) > 1e-6:
                        adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
                    else:
                        adjusted_prev[asset] = 0.0
            
            # Step 2: Calculate turnover
            all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
            
            turnover = 0.0
            for asset in all_assets:
                old_w = adjusted_prev.get(asset, 0.0)
                new_w = common_weights.get(asset, 0.0)
                turnover += abs(new_w - old_w)
            
            # Transaction cost on end-of-period portfolio value
            tc = transaction_cost * (1 + gross_return) * turnover
        
        else:
            # First period: no previous positions, buying into everything
            turnover = sum(abs(w) for w in common_weights.values())
            tc = transaction_cost * (1 + gross_return) * turnover
        
        # Calculate transaction costs - MV
        if len(prev_weights_dict_2) > 0:
            adjusted_prev_2 = {}
            
            for asset, prev_w in prev_weights_dict_2.items():
                if asset in prev_oos_returns_dict_2:
                    prev_r = prev_oos_returns_dict_2[asset]
                    if abs(1 + prev_gross_return_2) > 1e-6:
                        adjusted_prev_2[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return_2)
                    else:
                        adjusted_prev_2[asset] = 0.0
                else:
                    if abs(1 + prev_gross_return_2) > 1e-6:
                        adjusted_prev_2[asset] = prev_w / (1 + prev_gross_return_2)
                    else:
                        adjusted_prev_2[asset] = 0.0
            
            all_assets_2 = set(adjusted_prev_2.keys()) | set(common_weights_2.keys())
            
            turnover_2 = 0.0
            for asset in all_assets_2:
                old_w = adjusted_prev_2.get(asset, 0.0)
                new_w = common_weights_2.get(asset, 0.0)
                turnover_2 += abs(new_w - old_w)
            
            tc_2 = transaction_cost * (1 + gross_return_2) * turnover_2
        else:
            turnover_2 = sum(abs(w) for w in common_weights_2.values())
            tc_2 = transaction_cost * (1 + gross_return_2) * turnover_2
        
        # Calculate transaction costs - MSR
        if len(prev_weights_dict_3) > 0:
            adjusted_prev_3 = {}
            
            for asset, prev_w in prev_weights_dict_3.items():
                if asset in prev_oos_returns_dict_3:
                    prev_r = prev_oos_returns_dict_3[asset]
                    if abs(1 + prev_gross_return_3) > 1e-6:
                        adjusted_prev_3[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return_3)
                    else:
                        adjusted_prev_3[asset] = 0.0
                else:
                    if abs(1 + prev_gross_return_3) > 1e-6:
                        adjusted_prev_3[asset] = prev_w / (1 + prev_gross_return_3)
                    else:
                        adjusted_prev_3[asset] = 0.0
            
            all_assets_3 = set(adjusted_prev_3.keys()) | set(common_weights_3.keys())
            
            turnover_3 = 0.0
            for asset in all_assets_3:
                old_w = adjusted_prev_3.get(asset, 0.0)
                new_w = common_weights_3.get(asset, 0.0)
                turnover_3 += abs(new_w - old_w)
            
            tc_3 = transaction_cost * (1 + gross_return_3) * turnover_3
        else:
            turnover_3 = sum(abs(w) for w in common_weights_3.values())
            tc_3 = transaction_cost * (1 + gross_return_3) * turnover_3
        
        # Net returns
        net_return = gross_return - tc
        net_return_2 = gross_return_2 - tc_2
        net_return_3 = gross_return_3 - tc_3
        
        # Store results - GMV
        portfolio_returns.append(net_return)
        portfolio_dates.append(current_date)
        portfolio_weights_list.append(common_weights.copy())
        portfolio_turnover_list.append(turnover)
        portfolio_gross_returns.append(gross_return)
        
        # Store results - MV
        portfolio_returns_2.append(net_return_2)
        portfolio_dates_2.append(current_date)
        portfolio_weights_list_2.append(common_weights_2.copy())
        portfolio_turnover_list_2.append(turnover_2)
        portfolio_gross_returns_2.append(gross_return_2)
        
        # Store results - MSR
        portfolio_returns_3.append(net_return_3)
        portfolio_dates_3.append(current_date)
        portfolio_weights_list_3.append(common_weights_3.copy())
        portfolio_turnover_list_3.append(turnover_3)
        portfolio_gross_returns_3.append(gross_return_3)
        
        # Update previous values for next iteration
        prev_weights_dict = common_weights.copy()
        prev_oos_returns_dict = {a: oos_returns_dict[a] for a in common_assets}
        prev_gross_return = gross_return
        
        prev_weights_dict_2 = common_weights_2.copy()
        prev_oos_returns_dict_2 = {a: oos_returns_dict[a] for a in common_assets_2}
        prev_gross_return_2 = gross_return_2
        
        prev_weights_dict_3 = common_weights_3.copy()
        prev_oos_returns_dict_3 = {a: oos_returns_dict[a] for a in common_assets_3}
        prev_gross_return_3 = gross_return_3
        
        if verbose:
            print(f"  GMV  - Gross: {gross_return:>8.5f} | Turnover: {turnover:>6.4f} | "
                  f"TC: {tc:>8.6f} | Net: {net_return:>8.5f}")
            print(f"  MV   - Gross: {gross_return_2:>8.5f} | Turnover: {turnover_2:>6.4f} | "
                  f"TC: {tc_2:>8.6f} | Net: {net_return_2:>8.5f}")
            print(f"  MSR  - Gross: {gross_return_3:>8.5f} | Turnover: {turnover_3:>6.4f} | "
                  f"TC: {tc_3:>8.6f} | Net: {net_return_3:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results ---
    results_df = pd.DataFrame({
        'date': portfolio_dates,
        'portfolio_return': portfolio_returns,
        'portfolio_gross_return': portfolio_gross_returns,
        'portfolio_weights': portfolio_weights_list,
        'portfolio_turnover': portfolio_turnover_list
    })
    results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
    
    results_df_2 = pd.DataFrame({
        'date': portfolio_dates_2,
        'portfolio_return': portfolio_returns_2,
        'portfolio_gross_return': portfolio_gross_returns_2,
        'portfolio_weights': portfolio_weights_list_2,
        'portfolio_turnover': portfolio_turnover_list_2
    })
    results_df_2['cumulative_return'] = (1 + results_df_2['portfolio_return']).cumprod() - 1
    
    results_df_3 = pd.DataFrame({
        'date': portfolio_dates_3,
        'portfolio_return': portfolio_returns_3,
        'portfolio_gross_return': portfolio_gross_returns_3,
        'portfolio_weights': portfolio_weights_list_3,
        'portfolio_turnover': portfolio_turnover_list_3
    })
    results_df_3['cumulative_return'] = (1 + results_df_3['portfolio_return']).cumprod() - 1
    
    # Helper function to compute metrics
    def compute_metrics(returns_list, turnover_list, results_df):
        if len(returns_list) > 0:
            mean_return = np.mean(returns_list)
            variance = np.var(returns_list, ddof=1)
            sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
            
            # Annualized metrics (monthly data)
            annual_return = mean_return * 12
            annual_volatility = np.sqrt(variance * 12)
            annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
            
            return {
                'mean_return': mean_return,
                'variance': variance,
                'sharpe_ratio': sharpe_ratio,
                'annual_return': annual_return,
                'annual_volatility': annual_volatility,
                'annual_sharpe_ratio': annual_sharpe,
                'total_return': results_df['cumulative_return'].iloc[-1],
                'avg_turnover': np.mean(turnover_list),
                'n_periods': len(returns_list),
                'n_zero_periods': sum(1 for r in returns_list if r == 0)
            }
        else:
            return {
                'mean_return': 0, 'variance': 0, 'sharpe_ratio': 0,
                'annual_return': 0, 'annual_volatility': 0, 'annual_sharpe_ratio': 0,
                'total_return': 0, 'avg_turnover': 0, 'n_periods': 0, 'n_zero_periods': 0
            }
    
    # Compute metrics for all three strategies
    metrics = compute_metrics(portfolio_returns, portfolio_turnover_list, results_df)
    metrics_2 = compute_metrics(portfolio_returns_2, portfolio_turnover_list_2, results_df_2)
    metrics_3 = compute_metrics(portfolio_returns_3, portfolio_turnover_list_3, results_df_3)
    
    return results_df, metrics, results_df_2, metrics_2, results_df_3, metrics_3

In [4]:
df = pd.read_csv('../green_cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = df.groupby('permno')['ret_excess'].shift(-1)

data_f=pd.read_csv('F-F_Research_Data_Factors.csv',sep=',')
data_f['Date']=pd.to_datetime(data_f['Date'], format="%Y%m")
data_f['Date']=data_f['Date']+pd.offsets.MonthEnd(0)
data_f = data_f.set_index('Date')
data_f = data_f[['Mkt-RF', 'SMB', 'HML', 'RF']].astype(float)

# Run backtest with yearly signals
results_df, metrics, results_df_2, metrics_2, results_df_3, metrics_3= backtest_dnn_finbert(
    df,
    test_start_date='2015-01-31',
    test_end_date='2024-04-30',
    lookback_window=180,
    transaction_cost=0.001,
    signals_path='../examples/monthly_signals_decay.csv',
    data_factor=data_f,
    verbose=True
)

print(f"\n GMV")
print(f"\nSharpe Ratio: {metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {metrics['total_return']:.4f}")
print(f"Average Turnover: {metrics['avg_turnover']:.4f}")

print(f"\n MV")
print(f"\nSharpe Ratio: {metrics_2['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics_2['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {metrics_2['total_return']:.4f}")
print(f"Average Turnover: {metrics_2['avg_turnover']:.4f}")

print(f"\n MSR")
print(f"\nSharpe Ratio: {metrics_3['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {metrics_3['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {metrics_3['total_return']:.4f}")
print(f"Average Turnover: {metrics_3['avg_turnover']:.4f}")

Creating ticker to permno mapping...
Mapped 1664 unique tickers to permnos
Loading FinBERT signals from ../examples/monthly_signals_decay.csv...
Loaded 54240 monthly signals
Signal distribution:
signal
hold    52823
sell      859
buy       558
Name: count, dtype: int64
STARTING BACKTEST WITH DNN-FM + FINBERT (GMV/MV/MSR)

[1/112] Date: 2015-01-31
  Window: 2000-01-31 to 2014-12-31
  FinBERT signals: 10 | Assets w/ data: 5
  Running POET (arch=5)...
  Computing GMV weights...
  Computing MV weights...
  Computing MSR weights...
  GMV  - Gross:  0.05293 | Turnover: 1.0000 | TC: 0.001053 | Net:  0.05188
  MV   - Gross:  0.05092 | Turnover: 1.0000 | TC: 0.001051 | Net:  0.04987
  MSR  - Gross:  0.05130 | Turnover: 1.0000 | TC: 0.001051 | Net:  0.05025

[2/112] Date: 2015-02-28
  Window: 2000-02-29 to 2015-01-31
  FinBERT signals: 4 | Assets w/ data: 2
  Running POET (arch=5)...
  Computing GMV weights...
  Computing MV weights...
  Computing MSR weights...
  GMV  - Gross:  0.00648 | Turnov

In [5]:
print(f"\n GMV")
print(f"Annualized Sharpe Ratio: {metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {metrics['mean_return']*12:.4f}")
print(f"Variance: {metrics['variance']*12:.4f}")
print(f"Avg Turnover: {metrics['avg_turnover']:.4f}")

print(f"\n MV")
print(f"Annualized Sharpe Ratio: {metrics_2['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {metrics_2['mean_return']*12:.4f}")
print(f"Variance: {metrics_2['variance']*12:.4f}")
print(f"Avg Turnover: {metrics_2['avg_turnover']:.4f}")

print(f"\n MSR")
print(f"Annualized Sharpe Ratio: {metrics_3['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {metrics_3['mean_return']*12:.4f}")
print(f"Variance: {metrics_3['variance']*12:.4f}")
print(f"Avg Turnover: {metrics_3['avg_turnover']:.4f}")


 GMV
Annualized Sharpe Ratio: 0.7444
Mean Return: 0.1289
Variance: 0.0300
Avg Turnover: 1.6274

 MV
Annualized Sharpe Ratio: 0.7323
Mean Return: 0.1710
Variance: 0.0545
Avg Turnover: 2.0463

 MSR
Annualized Sharpe Ratio: 0.8063
Mean Return: 0.1345
Variance: 0.0278
Avg Turnover: 1.6400
